In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('/home/aliche/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [2]:
%matplotlib 
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import Recordings
import signale,trajectory
import extras.trees
import os,sys
import fnmatch
import colormaps as cmaps
import seaborn as sns
import scipy.signal as scsig

Using matplotlib backend: Qt5Agg


/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/NeuroTools/analysis.py:34: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to u'Qt5Agg' by the following code:
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages

In [3]:
def locate(pattern, root=os.curdir):
    '''
    Locate all files matching supplied filename pattern in and below
        supplied root directory.
    Parameters
    ----------
        pattern : string
            A string representing pattern you want to look for in the path.
        root : string
            contains the path address you want to look trough!
    Returns
    ----------
        Array of strings consist of two column, first contains the path and second the file names that matched 
        the desired pattern.
    '''
    for path, dirs, files in os.walk(os.path.abspath(root)):
        for filename in fnmatch.filter(files, pattern):
            yield [path,filename]
def ncs_loader(fileName):
    ncs_dtype = np.dtype([
        ('qTimeStamps'  ,     '<u8'),
        ('ChannelNumber'  , '<u4'),
        ('SampleFreq',      '<u4'),
        ('NumValidSamples', '<u4'),
        ('Samples'  ,       '<i2',(512,)),
    ])
    return np.memmap(fileName,dtype=ncs_dtype,offset=(16 * 2**10))
def nvt_loader(filename):
    """ 
    Memory map the Neuralynx .nvt format
    Fields
    -------
    swstx
    swid
    sw_data_size
    qTimeStamps       Cheetah timestamp for this record. This value is in microseconds.
    dwpoints          Points with the color bitfield values for this record.This is a 400
                      element array.  See Video Tracker in reference. 
    sncrc
    dnextracted_x     Extracted X location of the target being tracked. 
    dnextracted_y     Extracted Y location of the target being tracked.
    dnextracted_angle The calculated head angle in degrees clockwise from the positive Y
                      axis. Zero will be assigned if angle tracking is disabled.
    dnTargets         Colored targets using the samebitfield format used by the dwPoints array.
                      Instead of transitions, the bitfield indicates the colors that make up 
                      each particular target and the center point of that target.  This is a 50
                      element array sorted by size from largest (index 0) to smallest(index 49).
                      A target value of 0 means that no target is present in thatindex location.
                      See Video Tracker Bitfield Information in reference. 
    
    Reference:
    ----------
        http://neuralynx.com/software/NeuralynxDataFileFormats.pdf
    """ 
    nev_dtype = np.dtype([
        ('swstx'              , '<i2'),
        ('swid'               , '<i2'),
        ('sw_data_size'       , '<i2'),
        ('qTimeStamps'        , '<u8'),
        ('dwPoints'           , '<u4',(400,)),
        ('sncrc'              , '<i2'),
        ('dnextracted_x'      , '<i4'),
        ('dnextracted_y'      , '<i4'),
        ('dnextracted_angle'  , '<i4'),
        ('dnTargets'          , '<i4',(50,)),
    ])
    return np.memmap(filename, dtype=nev_dtype, mode='readwrite',
       offset=(16 * 2**10))
def nev_loader(filename):
    """ Memory map the Neuralynx .nev format """ 
    nev_dtype = np.dtype([
        ('nstx'           , '<i2'),
        ('npkt_id'        , '<i2'),
        ('npkt_data_size' , '<i2'),
        ('qTimeStamps'    , '<u8'),
        ('nevent_id'      , '<i2'),
        ('nttl'           , '<i2'),
        ('ncrc'           , '<i2'),
        ('ndummy1'        , '<i2'),
        ('ndummy2'        , '<i2'),
        ('dnExtra'        , '<i4',(8,)),
        ('String'         , '<S128')
    ])
    return np.memmap(filename, dtype=nev_dtype, mode='r',
       offset=(16 * 2**10))
###################################################### initialization
###################################################### Parameters
expType = 'real'
prefix = 'TT'
suffix = ''
noSpeck = False
onlyRunning = True
showHeadDir = False
saveFigs = True
useRecommended = False
TTName = '.t'

# initialize in order to make them available globally
spikes=[]
ID=-1
stList=signale.spikezugList(t_start=None, t_stop=None, dims=[2])
eventData = None

cwd=os.getcwd()


###################################################### functions

def getData(folderName):
    global spikes, ID, eventData

    if os.path.isdir(folderName):
        dirList=os.listdir(folderName)
        os.chdir(folderName)
    else:
        dirList = [folderName]
    for item in sorted(dirList):
        if os.path.isfile(item):
            if (TTName.__class__ == list and item in TTName) or\
                    (TTName.__class__ == str and item.endswith(suffix+'.t') and item.startswith(prefix)):
                print 'loading', item , 'from folder: '+'/'.join(folderName.split('/')[-3:])
                spikes = signale.load_tFile(item, showHeader=False)
                ID += 1
                stList.__setitem__(ID, spikes)
                stList.addTags(ID, file=item, dir=folderName)
            
        elif os.path.isdir(item):
            getData(item)
    os.chdir('..')


In [4]:
WorkPath = '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/'
animalPaths = ['/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/']

In [5]:
for pth in animalPaths:
    animal = pth.split('/')[-2]
    sessPaths = sorted([os.path.join(pth,item) for item in os.listdir(pth) if os.path.isdir(os.path.join(pth,item))])
    for dd,sess in enumerate(sessPaths[1:2]):
        dday = 'D'+str(dd+1)
        print animal+'-----'+dday
        parrentFolder = os.path.join(sess,'parrentData/')
        #print parrentFolder
        lfpList = sorted([os.path.join(item[0],item[1]) for item in locate('CSC1-*.lfp',sess)])
        print lfpList
        #lfpPRE,lfpTRACK,lfpPOST 
        lfpStack = [pkl.load(open(lfpFile,'rb')) for lfpFile in lfpList]
        for lfp in lfpStack:#[lfpPRE,lfpTRACK,lfpPOST]:
            lfp.filter(5,10)
        thetaTimeBinEdges = []
        for lfp in lfpStack:# [lfpPRE,lfpTRACK,lfpPOST]:
            thetaTimeBinEdges.append(lfp.timeAxis[scsig.argrelmax(lfp.signal_filtered)[0]])
        #pkl.dump(thetaTimeBinEdges,open(parrentFolder+'thBins.npArr','wb'),pkl.HIGHEST_PROTOCOL)
        tInterp = []
        for item in thetaTimeBinEdges:
            t = item
            It = np.arange(0,t.size)
            Itp = np.arange(0,t.size,0.25)
            tInterp.append(np.interp(Itp,It,t))
        #pkl.dump(tInterp,open(parrentFolder+'4thBins.npArr','wb'),pkl.HIGHEST_PROTOCOL)

Rat3661-----D1
['/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-04-15Day2/01-sleep1/CSC1-pre.lfp', '/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-04-15Day2/02-begin1/CSC1-track.lfp', '/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-04-15Day2/03-sleep2/CSC1-post.lfp']


In [50]:
rr = pkl.load(open('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3906-Day1-theta.Rl','rb'))[1]
thetaTime = pkl.load(open('/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/parrentData/thBins.npArr','rb'))

In [51]:
for ii,item in enumerate(rr[0]):
    pl.plot(item,label=str(ii))
pl.legend()

In [19]:
spk = pkl.load(open('/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/parrentData/FiringUnits.spk','rb')) 

In [6]:
lfp.timeAxis

array([ 6674432. ,  6674432.5,  6674433. , ..., 10281982.5, 10281983. ,
       10281983.5])

In [18]:
lfp.timeAxis[-1],thetaTimeBinEdges[2][-1]

(10281983.5, 10281962.5)

In [20]:
spk.time_axis(1)

array([  21291.09960938,   21292.09960938,   21293.09960938, ...,
       8798598.09960938, 8798599.09960938, 8798600.09960938])

In [24]:
[len(item) for item in thetaTimeBinEdges]

[37203, 14759, 28111]